## Estadistica Aplicada

Tarea 6

Bárbara Govea Serna 1902869

Grupo 42

Cada vez que un empleado renuncia a una empresa “X”, a esta empresa le toma como mínimo  6  meses  contratar  un  remplazo,  más  3  meses  de  capacitación  y  un  mes  de adaptación.  Tal  situación  implica pérdida de tiempo  y recursos  por  los  siguientes motivos:

a) Cuando  un  empleado  se  va,  sus  responsabilidades  deben  ser  redistribuidas entre  los  empleados  restantes.  Esto  puede  generar  una  carga  de  trabajo adicional para aquellos que quedan, lo que puede llevar a la fatiga 

b) Reemplazar a un empleado puede ser costoso. Incluye gastos en reclutamiento, selección, capacitación y adaptación

c) La rotación frecuente de empleados puede llevar a interrupciones en el flujo de trabajo,  ya  que  los  nuevos  empleados  necesitan  tiempo  para  adaptarse  y alcanzar el mismo nivel de productividad que sus predecesores

Por tal razón la empresa “X” desea conocer las razones que hacen que sus empleados renuncien,    para    tal    fin    le    ha    proporcionado una    base    de    datos llamada deserción_laboral que mide el tiempo en semestres desde que un empleado entró a trabajar a la empresa“X”hasta que renuncia. 
Donde el evento es renunciar.

Dada la información proporcionada el modelo más adecuado es el modelo de Cox de riesgos proporcionales. En una notebook realice lo siguiente:

## 1.Importa las librerías básicas y especializadas que consideres necesarias

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
from lifelines import CoxPHFitter,ExponentialFitter,WeibullFitter,LogNormalFitter


## 2. Importar la base de datos

In [4]:
desercion_laboral = pd.read_csv('https://raw.githubusercontent.com/jimmyzac/Estadistica-Aplicada-FCFM-UANL/main/bases_datos/desercion_laboral.csv')
desercion_laboral.head()

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,departamento,salario
0,3.8,5.3,2,157,3,0,1,0,ventas,bajo
1,8.0,8.6,5,262,6,0,1,0,ventas,medio
2,1.1,8.8,7,272,4,0,1,0,ventas,medio
3,7.2,8.7,5,223,5,0,1,0,ventas,bajo
4,3.7,5.2,2,159,3,0,1,0,ventas,bajo


Satisfaccion: Que tan satisfeho se encuentra la persona con su trabajo en una escala del 0 al 10

Evaluación: Que tan bien realiza su trabajo la persona en una escala del 0 al 10

Proyectos: Numero de proyectos asignados al empleado (2-5)

Promedio_horas_mes: Promedio de horas mensuales trabajadas

Semestre: Semestres que ha trabajado en la empresa (2-5)

Accidente: 1 Si ha tenido un accidente, 0 si no lo ha tenido (dentro de la empresa)

Renunciar: 1 si renuncio, 0 censurado

Ascenso_5years: 1 si el empleado ha sido ascendido durante los ultimos 5 años, 0 si no 

Departamento: Nombre del departamento en el que trabaja 

Salario: Clasificación del salario en alto, medio y bajo


## 3.Depurar la base de datos y estadísticas descriptivas
### a) Verificar que no haya missing values, en caso de haberlos decidir imputar o eliminar

In [5]:
## contar los valores perdidos/faltantes (missing values)
desercion_laboral.isnull().sum()

satisfaccion           0
evaluacion             0
proyectos              0
promedio_horas_mes     0
semestre               0
accidente              0
renunciar              0
ascenso_5years         0
departamento          26
salario                0
dtype: int64

In [6]:
# Elegimos eliminar con valores perdidos
desercion_laboral = desercion_laboral.dropna()

### b. Verificar que no haya valores duplicados, en caso de haber valores duplicados eliminarlos con el metodo dado

In [7]:
desercion_laboral.duplicated().sum()

2982

In [8]:
# Utilizando: dataframe=dataframe.drop_duplicates(keep='first').reset_index(drop=True)
desercion_laboral = desercion_laboral.drop_duplicates(keep='first').reset_index(drop=True)
# Verificamos que no haya mas
desercion_laboral.duplicated().sum()

0

### c. Verificar  que  todas  las  variables  sean  numéricas,  si  no  son  numéricas hacerlas   numéricas,   en   el   caso   de   las   categóricas   convertirlas   en dummies y conservar n-1 dummies

In [9]:
desercion_laboral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11991 entries, 0 to 11990
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   satisfaccion        11991 non-null  float64
 1   evaluacion          11991 non-null  float64
 2   proyectos           11991 non-null  int64  
 3   promedio_horas_mes  11991 non-null  int64  
 4   semestre            11991 non-null  int64  
 5   accidente           11991 non-null  int64  
 6   renunciar           11991 non-null  int64  
 7   ascenso_5years      11991 non-null  int64  
 8   departamento        11991 non-null  object 
 9   salario             11991 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 936.9+ KB


In [10]:
# Verificar que posibles opciones se encuentran en nuestra variable departamento
desercion_laboral['departamento'].unique()

array(['ventas', 'contabilidad', 'recursos humanos', 'mantenimiento',
       'soporte', 'administración', 'mercadotecnia', 'informática',
       'desarrollo de producto', 'Investigación'], dtype=object)

In [11]:
# Verificar que posibles opciones se encuentran en nuestra variable salario
desercion_laboral['salario'].unique()


array(['bajo', 'medio', 'alto'], dtype=object)

In [12]:
# Utilizamos drop_first=True para quedarnos con n-1 dummies, elimina la ultima en orden del array
desercion_laboral = pd.get_dummies(desercion_laboral,columns=['departamento','salario'], drop_first=True, dtype = int) 
# En departamento elimina 'Investigación', en salario elimina 'alto'

### d) Calcular estadísticas descriptivas

In [13]:
desercion_laboral.describe()

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,departamento_administración,departamento_contabilidad,departamento_desarrollo de producto,departamento_informática,departamento_mantenimiento,departamento_mercadotecnia,departamento_recursos humanos,departamento_soporte,departamento_ventas,salario_bajo,salario_medio
count,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.00000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000
mean,6.296581,7.166825,3.802852,200.473522,3.364857,0.154282,0.166041,0.016929,0.036361,0.051789,0.057210,0.081394,0.18714,0.056125,0.050121,0.151864,0.270119,0.478692,0.438746
std,2.410700,1.683426,1.163238,48.727813,1.330240,0.361234,0.372133,0.129012,0.187194,0.221610,0.232252,0.273451,0.39004,0.230173,0.218204,0.358904,0.444040,0.499567,0.496254
min,0.900000,3.600000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.800000,5.700000,3.000000,157.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.600000,7.200000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.200000,8.600000,5.000000,243.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
max,10.000000,10.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## 4. Estimar el modelo de Cox e interpretar cada uno de los coeficientes

In [16]:
cph1 = CoxPHFitter().fit(desercion_laboral,'semestre','renunciar')
cph1.print_summary()

ImportError: Missing optional dependency 'Jinja2'. DataFrame.style requires jinja2. Use pip or conda to install Jinja2.

<lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
      baseline estimation = breslow
   number of observations = 11991
number of events observed = 1991
   partial log-likelihood = -15884.05
         time fit was run = 2023-09-25 07:06:19 UTC

---
                                      coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                   
satisfaccion                         -0.24       0.78       0.01            -0.26            -0.22                 0.77                 0.80
evaluacion                           -0.01       0.99       0.01            -0.04             0.02                 0.96                 1.02
proyectos                            -0.29       0.75       0.02            -0.33            -0.25                 0.72                 0.78
promedio_horas_mes                    0.00       1.00       0.00             0.00             0.00                 1.00                 1.00
accidente                            -1.25       0.29       0.10            -1.44            -1.05                 0.24                 0.35
ascenso_5years                       -1.38       0.25       0.36            -2.08            -0.68                 0.13                 0.51
departamento_administración           0.22       1.25       0.18            -0.13             0.57                 0.88                 1.76
departamento_contabilidad             0.24       1.27       0.14            -0.04             0.52                 0.96                 1.69
departamento_desarrollo de producto   0.36       1.43       0.14             0.08             0.64                 1.08                 1.90
departamento_informática              0.29       1.34       0.13             0.03             0.56                 1.03                 1.74
departamento_mantenimiento            0.43       1.54       0.12             0.20             0.67                 1.22                 1.95
departamento_mercadotecnia            0.34       1.40       0.14             0.06             0.62                 1.06                 1.86
departamento_recursos humanos         0.43       1.54       0.14             0.15             0.72                 1.16                 2.05
departamento_soporte                  0.45       1.57       0.12             0.21             0.69                 1.23                 2.00
departamento_ventas                   0.35       1.42       0.12             0.12             0.58                 1.13                 1.78
salario_bajo                          1.54       4.66       0.15             1.25             1.83                 3.48                 6.23
salario_medio                         1.15       3.15       0.15             0.85             1.44                 2.35                 4.22

                                      cmp to      z      p   -log2(p)
covariate                                                            
satisfaccion                            0.00 -26.89 <0.005     526.58
evaluacion                              0.00  -0.76   0.44       1.17
proyectos                               0.00 -14.75 <0.005     161.15
promedio_horas_mes                      0.00   3.88 <0.005      13.25
accidente                               0.00 -12.41 <0.005     115.04
ascenso_5years                          0.00  -3.88 <0.005      13.21
departamento_administración             0.00   1.24   0.21       2.22
departamento_contabilidad               0.00   1.66   0.10       3.36
departamento_desarrollo de producto     0.00   2.49   0.01       6.29
departamento_informática                0.00   2.17   0.03       5.06
departamento_mantenimiento              0.00   3.60 <0.005      11.64
departamento_mercadotecnia          

In [ ]:
cph1.params_ 

covariate
satisfaccion                          -0.242337
evaluacion                            -0.010687
proyectos                             -0.291606
promedio_horas_mes                     0.001839
accidente                             -1.246686
ascenso_5years                        -1.380084
departamento_administración            0.220285
departamento_contabilidad              0.240389
departamento_desarrollo de producto    0.360015
departamento_informática               0.292078
departamento_mantenimiento             0.431970
departamento_mercadotecnia             0.338849
departamento_recursos humanos          0.434338
departamento_soporte                   0.451085
departamento_ventas                    0.350547
salario_bajo                           1.538901
salario_medio                          1.147612
Name: coef, dtype: float64

In [ ]:
cph1.hazard_ratios_

covariate
satisfaccion                           0.784792
evaluacion                             0.989370
proyectos                              0.747063
promedio_horas_mes                     1.001841
accidente                              0.287456
ascenso_5years                         0.251557
departamento_administración            1.246432
departamento_contabilidad              1.271743
departamento_desarrollo de producto    1.433351
departamento_informática               1.339208
departamento_mantenimiento             1.540290
departamento_mercadotecnia             1.403331
departamento_recursos humanos          1.543940
departamento_soporte                   1.570015
departamento_ventas                    1.419844
salario_bajo                           4.659468
salario_medio                          3.150662
Name: exp(coef), dtype: float64

### Interpretación de coeficientes: sea alpha = .05

- **satisfaccion:** p valor < .005 entonces p valor < alpha rechazamos H0, coef estadisticamente sig

El nivel de satisfacción de la persona en el trabajo disminuye el riesgo de renunciar un 22% de renunciar por cada unidad que aumente.

- **evaluación:** p valor = .44 entonces p valor > alpha, no rechazamos H0, coef no estadisticamente sig

La calificación de evaluación no afecta en el riesgo de renuncia, no aumenta ni disminuye

- **proyectos** p valor < .005 entonces p valor < alpha, rechazamos H0, coef estadisticamente sig

El riesgo de renunciar disminuye un 25% por cada proyecto que se le asigna a la persona

- **prom_horas_mes** p valor < .005 entonces p valor < alpha, rechazamos H0, coef estadisticamente sig

Entre mayor sea el promedio de horas trabajadas, el riesgo de renunciar aumento un .18% por cada hora trabajada

- **accidente** p valor < .005 entonces p valor < alpha, rechazamos H0, coef estadisticamente sig

Si la persona ha tenido un accidente en la empresa, el riesgo de renunciar disminuye un 71% en comparación con los que no han tenido. 

- **ascenso 5_years:** p valor < .005 entonces p valor < alpha, rechazamos H0, coef estadisticamente sig

Si la persona ha ascendido durante los ultimos 5 años, el riesgo de renunciar disminuye un 75% en comparación que los que no han ascendido durante los ultimos 5 años. 

- **departamento_administración, contabilidad** valor > alpha, no rechazamos H0, coef no estadisticamente sig

Que la persona trabaje en el departamento de administración o contabilidad no afecta en el riesgo de renunciar de una persona, no aumenta ni disminuye en comparación con el que trabaja en el departamento de investigación

- **departamento_desarrollo de producto, informatica, mantenimiento, mercadotecnia, recursos humanos, soporte, ventas**  p valor < alpha, rechazamos h0, coef estadisticamente sig

-- Que la persona trabaje en el departamento de desarrollo de producto, aumenta el riesgo de renunciar un 43% en comparación con el que trabaja en el departamento de investigación

-- Que la persona trabaje en el departamento de informatica, aumenta el riesgo de renunciar un 34% en comparación con el que trabaja en el departamento de investigación

-- Que la persona trabaje en el departamento de mantenimiento, aumenta el riesgo de renunciar un 54% en comparación con el que trabaja en el departamento de investigación

-- Que la persona trabaje en el departamento de mercadotecnia, aumenta el riesgo de renunciar un 40% en comparación con el que trabaja en el departamento de investigación

-- Que la persona trabaje en el departamento de desarrollo de recursos humanos, aumenta el riesgo de renunciar un 54% en comparación con el que trabaja en el departamento de investigación

-- Que la persona trabaje en el departamento de soporte, aumenta el riesgo de renunciar un 57% en comparación con el que trabaja en el departamento de investigación

-- Que la persona trabaje en el departamento de ventas, aumenta el riesgo de renunciar un 42% en comparación con el que trabaja en el departamento de investigación

- **salario_bajo,medio**  p valor < alpha, rechazamos h0, coef estadisticamente sig

-- Que la persona cuente con un salario bajo aumenta el riesgo de renunciar un 366% en comparación el que cuenta con un salario alto 

-- Que la persona cuente con un salario medio aumenta el riesgo de renunciar un 215% en comparación el que cuenta con un salario alto 



## 5. Con base  en la interpretación de los coeficientes,¿qué recomendación podría hacerle  a  la  empresa  para  que  disminuya  la  cantidad  de  renuncias  de  los empleados?


Debe prestar atención a aquellas variables que hacen que el riesgo de renunciar aumente, como lo es el promedio de horas trabajadas, por cada hora trabajada se incrementa el riesgo por lo que es de suma importancia no sobrecargar al empleado. Además, se prestar atención a los departamentos de desarrollo de producto, informatica, manteniiento, mercadotecnia, desarrollo de recursos humanos, soporte y ventas ya que se aumenta el riesgo en comparación con el departamento de investigación; buscar las razones que hacen que este riesgo aumenta y atacar el problema para que el riesgo disminuya. 

Además, notamos que el riesgo de renunciar aumenta notablemente cuando el empleado cuenta con un salario bajo y medio, por arriba del 100% 



## 6. Pruebe si el modelo de Cox cumple el supuesto de riesgos proporcionales, de no hacerlo estime el modelo AFT adecuado

Para verificar si se cumple el supuesto de riesgos proporcionales se cumple hacemos un contraste de hipotesis, donde

$H0$ : No se cumple el supuesto de riesgo proporcional constante

$H1$ : Se cumple el supuesto de riesgo proporcional constante

Si p valor < alpha, rechazamos h0

Si p valor > alpha, aceptamos h0

In [15]:
cph1.check_assumptions(desercion_laboral,p_value_threshold=.05)

The ``p_value_threshold`` is set at 0.05. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.



ImportError: Missing optional dependency 'Jinja2'. DataFrame.style requires jinja2. Use pip or conda to install Jinja2.

<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
         test_name = proportional_hazard_test

---
                                          test_statistic      p  -log2(p)
accidente                           km              0.03   0.87      0.21
                                    rank            1.14   0.29      1.81
ascenso_5years                      km              3.66   0.06      4.17
                                    rank            6.71   0.01      6.71
departamento_administración         km              5.08   0.02      5.37
                                    rank            5.74   0.02      5.92
departamento_contabilidad           km              0.09   0.77      0.39
                                    rank            1.14   0.29      1.81
departamento_desarrollo de producto km              1.03   0.31      1.69
                                    rank            0.85   0.36      1.49
departamento_informática            km              0.48   0.49      1.03
                                    rank            0.53   0.47      1.10
departamento_mantenimiento          km              0.47   0.49      1.02
                                    rank            2.26   0.13      2.91
departamento_mercadotecnia          km              2.66   0.10      3.28
                                    rank            6.33   0.01      6.39
departamento_recursos humanos       km              0.68   0.41      1.29
                                    rank            2.86   0.09      3.46
departamento_soporte                km              0.00   0.98      0.03
                                    rank            0.89   0.35      1.53
departamento_ventas                 km              1.04   0.31      1.70
                                    rank            4.36   0.04      4.76
evaluacion                          km            443.05 <0.005    324.32
                                    rank          427.15 <0.005    312.82
promedio_horas_mes                  km            270.84 <0.005    199.74
                                    rank          307.41 <0.005    226.21
proyectos                           km            574.13 <0.005    419.05
                                    rank          684.93 <0.005    499.11
salario_bajo                        km              0.37   0.54      0.88
                                    rank            0.00   0.96      0.06
salario_medio                       km              0.16   0.69      0.53
                                    rank            0.10   0.75      0.42
satisfaccion                        km           1257.74 <0.005    912.74
                                    rank          845.24 <0.005    614.90



1. Variable 'satisfaccion' failed the non-proportional test: p-value is <5e-05.

   Advice 1: the functional form of the variable 'satisfaccion' might be incorrect. That is, there
may be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'satisfaccion' using pd.cut, and then specify it in
`strata=['satisfaccion', ...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


2. Variable 'evaluacion' failed the non-proportional test: p-value is <5e-05.

   Advice 1: the functional form of the variable 'evaluacion' might be incorrect. That is, there may
be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to 

[]

Algunas variables no cumplen el supuesto de riesgo proporcional constante, utilizaremos el modelo alternativo AFT

## 7. Interprete los coeficientes del modelo AFT

In [20]:
# Para usar este modelo, tenemos que verificar si es Weibull, Exponencial o Lognormal. 
expo = ExponentialFitter().fit(desercion_laboral['semestre'],desercion_laboral['renunciar'])
mweibull = WeibullFitter().fit(desercion_laboral['semestre'],desercion_laboral['renunciar'])
mlogn = LogNormalFitter().fit(desercion_laboral['semestre'],desercion_laboral['renunciar'])
expo.AIC_,mweibull.AIC_,mlogn.AIC_


(15965.45892201228, 12511.512867661068, 11587.92169210356)

#### Elegimos el menor AIC, osea el LogNormal


In [21]:
from lifelines import LogNormalAFTFitter

In [24]:
mlognAFT = LogNormalAFTFitter().fit(desercion_laboral,'semestre','renunciar')
mlognAFT.print_summary()

ImportError: Missing optional dependency 'Jinja2'. DataFrame.style requires jinja2. Use pip or conda to install Jinja2.

<lifelines.LogNormalAFTFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
   number of observations = 11991
number of events observed = 1991
           log-likelihood = -4876.57
         time fit was run = 2023-09-25 17:16:11 UTC

---
                                             coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
param  covariate                                                                                                                                   
mu_    accidente                             0.26       1.30       0.02             0.22             0.30                 1.25                 1.35
       ascenso_5years                        0.37       1.44       0.06             0.24             0.49                 1.27                 1.63
       departamento_administración           0.05       1.05       0.04            -0.03             0.13                 0.97                 1.13
       departamento_contabilidad            -0.04       0.96       0.03            -0.11             0.02                 0.90                 1.02
       departamento_desarrollo de producto  -0.05       0.95       0.03            -0.12             0.01                 0.89                 1.01
       departamento_informática             -0.05       0.95       0.03            -0.11             0.01                 0.90                 1.02
       departamento_mantenimiento           -0.08       0.93       0.03            -0.13            -0.02                 0.88                 0.98
       departamento_mercadotecnia           -0.04       0.97       0.03            -0.10             0.03                 0.90                 1.03
       departamento_recursos humanos        -0.09       0.92       0.03            -0.15            -0.02                 0.86                 0.98
       departamento_soporte                 -0.08       0.92       0.03            -0.14            -0.03                 0.87                 0.97
       departamento_ventas                  -0.06       0.94       0.03            -0.11            -0.00                 0.90                 1.00
       evaluacion                            0.01       1.01       0.00            -0.00             0.01                 1.00                 1.01
       promedio_horas_mes                   -0.00       1.00       0.00            -0.00            -0.00                 1.00                 1.00
       proyectos                             0.10       1.10       0.01             0.09             0.11                 1.09                 1.11
       salario_bajo                         -0.37       0.69       0.03            -0.43            -0.31                 0.65                 0.73
       salario_medio                        -0.29       0.75       0.03            -0.34            -0.23                 0.71                 0.80
       satisfaccion                          0.07       1.07       0.00             0.06             0.07                 1.06                 1.07
       Intercept                             1.34       3.83       0.04             1.25             1.43                 3.50                 4.18
sigma_ Intercept                            -1.11       0.33       0.02            -1.14            -1.08                 0.32                 0.34

                                             cmp to      z      p   -log2(p)
param  covariate                                                            
mu_    accidente                               0.00  12.98 <0.005     125.59
       ascenso_5years                          0.00   5.80 <0.005      27.19
       departamento_administración             0.00   1.21   0.23       2.14
       departamento_contabilidad               0.00  -1.27   0.20       2.30
       departamento_desarrollo de producto     0.00  -1.54   0.12       3.01
       d

In [28]:
mlognAFT.params_

param   covariate                          
mu_     accidente                              0.263048
        ascenso_5years                         0.365029
        departamento_administración            0.047968
        departamento_contabilidad             -0.043292
        departamento_desarrollo de producto   -0.051817
        departamento_informática              -0.046400
        departamento_mantenimiento            -0.077644
        departamento_mercadotecnia            -0.035578
        departamento_recursos humanos         -0.086226
        departamento_soporte                  -0.083025
        departamento_ventas                   -0.057362
        evaluacion                             0.005721
        promedio_horas_mes                    -0.000309
        proyectos                              0.096128
        salario_bajo                          -0.369968
        salario_medio                         -0.286107
        satisfaccion                           0.067044
    

In [41]:
# Ver exp(coef), con decimales de promedio_horas_mes
import math
exp_coef = math.exp(-0.000309)
1 - exp_coef

0.00030895226441685075

### Interpretación de coeficientes: sea alpha = .05

- **accidente, ascenso 5_years** p valor < .005 entonces p valor < alpha, rechazamos H0, coef estadisticamente sig

-- Si la persona ha tenido un accidente en la empresa, la mediana del tiempo de supervivencia aumenta un 30% por cada accidente en comparación con los que no han tenido un accidente. 

-- Si la persona ha ascendido durante los ultimos 5 años, la mediana del tiempo de supervivencia aumenta un 44% en comparación que los que no han ascendido durante los ultimos 5 años. 

- **departamento_administración, contabilidad, desarrollo de producto, informatica, mercadotecnia** p valor > alpha, no rechazamos H0, coef no estadisticamente sig

Que la persona trabaje en el departamento de administración, contabilidad, desarrollo de producto, informatica o mercadotecnia no afecta la mediana del tiempo de supervivencia, no aumenta ni disminuye en comparación con el que trabaja en el departamento de investigación

- **departamento_mantenimiento** p valor < alpha, entonces rechazamos H0, coef estadisticamente sig

Que la persona trabaje en el departamento de mantenimiento disminuye un 7% la mediana del tiempo de supervivencia en comparación con el que trabaja en el departamento de investigación. 

- **departamento_recursos humanos** p valor < alpha, entonces rechazamos H0, coef estadisticamente sig

Que la persona trabaje en el departamento de recursos humanos disminuye un 8% la mediana del tiempo de supervivencia en comparación con el que trabaja en el departamento de investigación.

- **departamento_soporte** p valor < .005 entonces p valor < alpha, rechazamos H0, coef estadisticamente sig

Que la persona trabaje en el departamento de soporte disminuye un 8% la mediana del tiempo de supervivencia en comparación con el que trabaja en el departamento de investigación.

- **evaluación** p valor > alpha, no rechazamos H0, coef no estadisticamente sig

La calificación de evaluación no afecta en la mediana del tiempo de supervivencia, no aumenta ni disminuye

- **promedio_horas_mes, proyectos, salario_bajo, salario_medio, satisfaccion** p valor < alpha, rechazamos H0, coef estadisticamente sig 

-- Entre mayor sea el promedio de horas trabajadas, la mediana del tiempo de supervivencia disminuye un .03% la mediana del tiempo de supervivencia por cada hora trabajada agregada

-- La mediana del tiempo de supervivencia aumenta 10% por cada proyecto que se le asigna a la persona

-- Que la persona cuente con un salario bajo disminuye la mediana del tiempo de supervivencia un 31% en comparación con el que cuenta un salario alto

-- Que la persona cuente con un salario medio disminuye la mediana del tiempo de supervivencia un 25% en comparación con el que cuenta un salario alto

-- El nivel de satisfacción de la persona en el trabajo aumenta la mediana del tiempo de supervivencia un 7% por cada unidad que aumente.
